## Questions 

For the following categories: 
1. fatalities
2. serious injuries
3. pedestrian and bicycle crashes
   
Answer the following questions for El Paso county from 2020-2024
- How many crashes?
- How many people were affected?
- Which intersections are most prevalent?


### Set Up

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

## Load Data

In [7]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format
from cdot_clean_geography_columns import handle_missing_values_in_geography_columns, \
fill_missing_city_values, create_truncated_lat_long, combine_loc1_loc2_alphabetically 

In [8]:
# Load CDOT Data for a list of years

years = [2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2020.xlsx
['SYSTEM', 'RTE', 'SEC', 'MP', 'DATE']
(86304, 85)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2021.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(97848, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2022.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(95431, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2023.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(102166, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(99980, 111)


## Prepare Data

### Limit to El Paso County

In [9]:
print('Loaded Data Size', cdot_pdf_load.shape)

cdot_pdf_elpaso = cdot_pdf_load.loc[cdot_pdf_load['County'] == 'EL PASO']

print('El Paso County Data Size', cdot_pdf_elpaso.shape)

Loaded Data Size (481729, 149)
El Paso County Data Size (49422, 149)


### Clean Data

In [27]:
# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_elpaso.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)
display(cdot_pdf_withNans_dedup.head(5))

# Select columns for analysis (ALL FOR NOW)  
analysis_columns = list(cdot_pdf_withNans_dedup.columns)
cdot_pdf = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
#print('nan count before handling') 
#for col in cdot_pdf.columns:
#    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
#    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
#print()

# handle nan, null, and other corner cases
cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf)

# check NaN count by column 
print('nan count after handling')
print('handle nans for only geography columns') 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

# Create truncated latitude and longitude for analysis 
cdot_pdf = create_truncated_lat_long(cdot_pdf, 3) 
print(cdot_pdf.columns)

# combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************
cdot_pdf = combine_loc1_loc2_alphabetically(cdot_pdf) 

# check result
display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

Data size after drop duplicates (49417, 149)


,System Code,Crash Date,Injury 00,Injury 01,Injury 02,Injury 03,Injury 04,Agency Id,Location,Road Description,...,Wild Animal,Road Contour Curves,Road Contour Grade,Number Killed,Number Injured,Approach Overtaking Turn,Crash Time,Rd_Number,Rd_Section,City_Street
9,City Street,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,31:00:00,NaN,VERDE,NaN
14,City Street,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,10:50:00,NaN,BOULD,NaN
16,County Road,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSP,Ran off right side,Non-Intersection,...,NaN,Curve Unknown Direction,Unknown,0.0,0.0,Not Applicable,11:50:00,NaN,MOUNT,NaN
18,Interstate Highway,2020-01-01,0.0,0.0,1.0,0.0,0.0,CSP,Ran off left side,Non-Intersection,...,NaN,Straight,Level,0.0,1.0,Not Applicable,12:50:00,25.0A,131.0,NaN
31,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Unknown,0.0,0.0,Not Applicable,30:00:00,NaN,BRIDG,NaN


nan count after handling
handle nans for only geography columns
System Code 0
System Code 0
Crash Date 0
Crash Date 0
Injury 00 0
Injury 00 0
Injury 01 0
Injury 01 0
Injury 02 0
Injury 02 0
Injury 03 0
Injury 03 0
Injury 04 0
Injury 04 0
Agency Id 1
Agency Id 1
Location 10
Location 10
Road Description 4
Road Description 4
Total Vehicles 16
Total Vehicles 16
Road Condition 25
Road Condition 25
Lighting Conditions 33
Lighting Conditions 33
Weather Condition 35
Weather Condition 35
First HE 1497
First HE 1497
Second HE 37959
Second HE 37959
Third HE 46680
Third HE 46680
MHE 1498
MHE 1498
Crash Type 2
Crash Type 2
Location 1 1
Location 1 1
Link 12
Link 12
Location 2 30
Location 2 30
City 0
City 0
County 0
County 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0
Crash Year 0
Crash Year 0
CUID 8251
CUID 8251
Fourth HE 48708
Fourth HE 48708
Secondary Crash 8266
Secondary Crash 8266
Construction Zone 8266
Construction Zone 8266
School Zone 8266
School Zone 8266
Weather Condition 2 46464
Weather 

,Location 1,Location 2,loc1_first,alph_loc1_loc2
9,VERDE DR,CHELTON RD S,False,CHELTON RD S VERDE DR
14,BOULDER ST E,NEVADA AVE N,True,BOULDER ST E NEVADA AVE N
16,MOUNTAIN PINE LN,ROLLER COASTER RD,True,MOUNTAIN PINE LN ROLLER COASTER RD
18,I-25,MM 131,True,I-25 MM 131
31,BRIDGEWOOD LN,CHELTON RD S,True,BRIDGEWOOD LN CHELTON RD S


### Crashes with Fatalities

- Injury 00 = persons not injured
- Injury 01 = persons with possible injury
- Injury 02 = persons with suspected minor injury
- Injury 03 = persons with suspected serious injury
- Injury 04 = persons killed
- Number Killed (equivalent to injury 4) 
- Number Injured (sum over injury 1, 2, 3)

In [48]:
# How many crashes with fatalities in El Paso County?
print(f"Number of crashes with fatalities in {years}:", cdot_pdf.loc[cdot_pdf['Number Killed'] > 0].count()['Crash Date'])

# How many fatalities from crashes?
print(f"Number of fatalities in {years}:", int(cdot_pdf['Number Killed'].sum()))

# Number of people involved in fatal crashes 
print(f"Number of people involved in fatal crashes in {years}:", int(cdot_pdf['Number Killed'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Number Killed'] > 0, 'Injury 00'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Number Killed'] > 0, 'Injury 01'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Number Killed'] > 0, 'Injury 02'].sum() +
                                            cdot_pdf.loc[cdot_pdf['Number Killed'] > 0, 'Injury 03'].sum()))



Number of crashes with fatalities in [2020, 2021, 2022, 2023, 2024]: 383
Number of fatalities in [2020, 2021, 2022, 2023, 2024]: 401
Number of people involved in fatal crashes in [2020, 2021, 2022, 2023, 2024]: 954


In [25]:
print('Number of lat long unique values with a fatal crash', 
      cdot_pdf.loc[cdot_pdf['Number Killed'] > 0].groupby(['Latitude_3dec', 'Longitude_3dec']).count()['CUID'].sort_values(ascending=False).shape[0])

# get the lat long combos with the highest number of crashes with one or more fatality
gb_latlong_fatal_crash_ct = cdot_pdf.loc[cdot_pdf['Number Killed'] > 0] \
                                    .groupby(['Latitude_3dec', 'Longitude_3dec']).count()['CUID'] \
                                    .sort_values(ascending=False).reset_index() \
                                    .rename(columns={'CUID':'crash_count'})

# get lat long values where more than one fatal crash occurred 
gb_ll_fatal_gr_1 = gb_latlong_fatal_crash_ct.loc[gb_latlong_fatal_crash_ct['crash_count'] >= 2]
display(gb_ll_fatal_gr_1)

Number of lat long unique values with a fatal crash 358


,Latitude_3dec,Longitude_3dec,crash_count
0,38.737,-104.683,3
1,38.815,-104.825,2
2,38.810,-104.809,2
3,38.944,-104.792,2
4,38.833,-104.785,2
5,38.934,-104.812,2
6,38.864,-104.664,2
7,38.858,-104.700,2
8,38.891,-104.802,2
9,39.029,-104.837,2


In [33]:
fatality_view_columns = ['Crash Date', 
                         'Crash Time', 
                         'Latitude', 
                         'Longitude', 
                         'Location 1', 
                         'Location 2', 
                         'Crash Type',
                         'Number Killed', 
                        ]

print(f"Intersections with more than one fatal crash in {years}")
for index, row in gb_ll_fatal_gr_1.iterrows():
    print()
    display(cdot_pdf.loc[(cdot_pdf['Number Killed'] > 0) & 
                         (cdot_pdf['Latitude_3dec'] == row['Latitude_3dec']) & 
                         (cdot_pdf['Longitude_3dec'] == row['Longitude_3dec']), fatality_view_columns])

Intersections with more than one fatal crash in [2020, 2021, 2022, 2023, 2024]



,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
87276,2022-12-04,17:12:00,38.737333,-104.682742,HWY 21,FONTAINE BLVD,Bicycle/Motorized Bicycle,1.0
9865,2023-02-03,19:56:00,38.737439,-104.682961,HWY 21,FONTAINE BLVD,Approach Turn,1.0
85872,2023-11-05,15:49:00,38.737480,-104.682740,HWY 21,FONTAINE BLVD,Approach Turn,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
51598,2022-07-18,12:19:00,38.815472,-104.824600,S TEJON ST,I-25 NB RAMPS,Approach Turn,1.0
81059,2023-10-20,19:24:00,38.814589,-104.824761,I-25 SB OFF RAMP,S TEJON ST,Pedestrian,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
40102,2024-05-26,04:47:00,38.81040,-104.80929,I-25,MM 138.982,Head-On,2.0
40103,2024-05-26,04:48:00,38.81035,-104.80921,I-25,MARTIN LUTHER KING JR BYPASS,Pedestrian,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
30329,2022-04-26,10:14:00,38.944422,-104.792294,CHAPEL HILLS DR,BRIARGATE PKWY,Approach Turn,2.0
40060,2024-05-25,21:00:00,38.944444,-104.792333,BRIARGATE PKWY,CHAPEL HILLS DR,Approach Turn,1.0


...

,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
62755,2021-09-02,13:07:00,38.89140,-104.80205,AUSTIN BLUFFS PKWY,REGENT CIR,Approach Turn,1.0
77658,2024-10-12,07:47:00,38.89136,-104.80190,AUSTIN BLUFFS PKWY,REGENT CIR,Broadside,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
54842,2023-07-21,20:06:00,39.029392,-104.837122,I-25,MM 156.2,Guardrail Face,1.0
78232,2024-10-14,16:25:00,39.029290,-104.837040,I-25 SB,MM 156.445,Rear-End,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
26674,2021-04-17,21:12:00,38.71919,-104.70109,MESA RIDGE PKWY,FOUNTAIN MESA RD,Pedestrian,2.0
19095,2024-03-07,00:12:00,38.71910,-104.70095,MESA RIDGE PKWY,FOUNTAIN MESA RD,Pedestrian,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
54993,2023-07-22,12:48:00,38.87614,-104.93568,HWY 24,MM 296.462,Guardrail Face,1.0
17400,2024-02-29,15:20:00,38.87603,-104.93636,HWY 24,MM 296.43,Guardrail Face,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
10942,2023-02-08,05:31:00,38.808692,-104.741494,JET WING DR,E FOUNTAIN BLVD,Pedestrian,1.0
56813,2023-07-29,00:07:00,38.808750,-104.741322,JET WING DR,E FOUNTAIN BLVD,Broadside,1.0


In [37]:
# get the lat long combos with the highest number fatalities

gb_latlong_fatality_ct = cdot_pdf.groupby(['Latitude_3dec', 'Longitude_3dec'])['Number Killed'].sum() \
                                    .sort_values(ascending=False).reset_index() \
                                    .rename(columns={'Number Killed':'fatality_count'})

# get lat long values where more than one fatal crash occurred 
gb_ll_fatality_gr_1 = gb_latlong_fatality_ct.loc[gb_latlong_fatality_ct['fatality_count'] >= 2]
display(gb_ll_fatality_gr_1)

for index, row in gb_ll_fatality_gr_1.iterrows():
    print()
    display(cdot_pdf.loc[(cdot_pdf['Number Killed'] > 0) & 
                         (cdot_pdf['Latitude_3dec'] == row['Latitude_3dec']) & 
                         (cdot_pdf['Longitude_3dec'] == row['Longitude_3dec']), fatality_view_columns])

,Latitude_3dec,Longitude_3dec,fatality_count
0,38.810,-104.809,3.0
1,38.944,-104.792,3.0
2,38.719,-104.701,3.0
3,38.737,-104.683,3.0
4,38.586,-104.917,2.0
5,38.806,-104.730,2.0
6,38.815,-104.825,2.0
7,39.042,-104.273,2.0
8,38.864,-104.664,2.0
9,38.999,-104.781,2.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
40102,2024-05-26,04:47:00,38.81040,-104.80929,I-25,MM 138.982,Head-On,2.0
40103,2024-05-26,04:48:00,38.81035,-104.80921,I-25,MARTIN LUTHER KING JR BYPASS,Pedestrian,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
30329,2022-04-26,10:14:00,38.944422,-104.792294,CHAPEL HILLS DR,BRIARGATE PKWY,Approach Turn,2.0
40060,2024-05-25,21:00:00,38.944444,-104.792333,BRIARGATE PKWY,CHAPEL HILLS DR,Approach Turn,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
26674,2021-04-17,21:12:00,38.71919,-104.70109,MESA RIDGE PKWY,FOUNTAIN MESA RD,Pedestrian,2.0
19095,2024-03-07,00:12:00,38.71910,-104.70095,MESA RIDGE PKWY,FOUNTAIN MESA RD,Pedestrian,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
87276,2022-12-04,17:12:00,38.737333,-104.682742,HWY 21,FONTAINE BLVD,Bicycle/Motorized Bicycle,1.0
9865,2023-02-03,19:56:00,38.737439,-104.682961,HWY 21,FONTAINE BLVD,Approach Turn,1.0
85872,2023-11-05,15:49:00,38.737480,-104.682740,HWY 21,FONTAINE BLVD,Approach Turn,1.0


...

,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
17964,2020-02-28,17:41:00,38.726197,-104.728222,HWY 16,BANDLEY DR,Guardrail Unspecified,1.0
25043,2020-04-05,22:54:00,38.726089,-104.727839,HWY 16,BANDLEY DR,Guardrail Unspecified,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
51700,2024-07-10,22:22:00,38.96087,-104.57067,HWY 24,MM 323.088,Head-On,2.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
33190,2020-06-02,21:11:00,38.879678,-104.756911,ACADEMY BLVD N,VILLAGE SEVEN RD,Pedestrian,1.0
28382,2021-04-24,21:39:00,38.879580,-104.757040,N ACADEMY BLVD,REBECCA LN,Approach Turn,1.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
77946,2024-10-13,11:35:00,38.97567,-104.55067,HWY 24,MM 324.515,Head-On,2.0


,Crash Date,Crash Time,Latitude,Longitude,Location 1,Location 2,Crash Type,Number Killed
14646,2021-02-28,09:35:00,39.013039,-104.663361,SHOUP RD,VOLLMER ROAD,Broadside,2.0


### Crashes with Serious Injuries

In [49]:
# How many crashes with serious injuries in El Paso County?
print(f"Number of crashes with serious injuries in {years}:", cdot_pdf.loc[cdot_pdf['Injury 03'] > 0].count()['Crash Date'])

# How many serious injuries from crashes?
print(f"Number of serious injuries in {years}:", int(cdot_pdf['Injury 03'].sum()))

# Number of people involved in serious injury crashes 
print(f"Number of people involved in fatal crashes in {years}:", int(cdot_pdf['Injury 03'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Injury 03'] > 0, 'Injury 00'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Injury 03'] > 0, 'Injury 01'].sum() + 
                                            cdot_pdf.loc[cdot_pdf['Injury 03'] > 0, 'Injury 02'].sum() +
                                            cdot_pdf.loc[cdot_pdf['Injury 03'] > 0, 'Injury 04'].sum()))

Number of crashes with serious injuries in [2020, 2021, 2022, 2023, 2024]: 1808
Number of serious injuries in [2020, 2021, 2022, 2023, 2024]: 2075
Number of people involved in fatal crashes in [2020, 2021, 2022, 2023, 2024]: 4876


### Pedestrian and Bicycle Crashes

- Crash Type = 'Bicycle/Motorized Bicycle', 'Pedestrian'

In [60]:
# How many crashes with pedestrian or bicycle crashes in El Paso County?
print(f"Number of crashes with a pedestrian or bicycle in {years}:")
print(cdot_pdf.loc[cdot_pdf['Crash Type'].isin(['Bicycle/Motorized Bicycle', 'Pedestrian'])].shape[0])

# TODO - number of bikers or pedestrians involved in crashes
# TODO - number of people involved in pedestrian or bicycle crashes
# TODO - number of people killed in pedestrian or bicycle crashes
# TODO - number of people seriously injured in pedestrain or bicycle crashes 

# Do we care to count the number of pedestrians and bicyclists injured? (in addition to the number of injuries)??


Number of crashes with a pedestrian or bicycle in [2020, 2021, 2022, 2023, 2024]:
1148
